# Python and OpenGL for Scientific Visualisation

These are notes and associated images are taken from the Python & OpenGL for Scientific book by Nicolas P. Rougier (available at: www.labri.fr/perso/nrougier/python-opengl)

## Modern OpenGL

### The Graphic Pipeline

Shaders are pieces of program that are built onto the GPU and carried out in the rendering pipeline. GLSL is the language used for shaders. The diagram below shows **vertex** and **fragment** shaders. The **vertex** shader is executed for every vertex inputted to the rendering pipeline and the **fragment** shader acts on each fragment/pixel genersted at the vertex stage. In the diagram below the **vertex** shader is called 3 times and the **fragment** is called 21 times.

![Graphic Pipeline](http://www.labri.fr/perso/nrougier/python-opengl/images/chapter-02/gl-pipeline.png)

**Vertex** shaders act on vertices and output the vertex position (`gl_Position`) on the screen. **Fragment** shaders output the colour (`gl_FragColor`) of the fragment.

A minimal vertex shader would be (this outputs the null vertex):

`void main()
{
    gl_Position = vec4(0.0,0.0,0.0,1.0);
}`

and a minimal fragment shader given by the following (outputs black colour):

`void main()
{
    gl_FragColor = vec4(0.0,0.0,0.0,1.0);
}`

### Buffers

But where do these vertices come from?

Vertices are stored on the CPU and need to be uploaded to the GPU before rendering. This is done by huilding buffers on the CPU and sending these to the GPU. These only need to be uploaded once if the data is static. This is different from the **fixed pipeline** where the data needed to be uploaded at each rendering call.

The structure of the vertices is mostly free but all the vertices from a buffer need to have the same stucture.

We will create an example buffer below of 4 vertices, each having associated x, y and z coordinates and a color:

`data = numpy.zeros(4, dtype = [("position", np.float32, 3)
    ("colour", np.float32, 4)]`

### Variables 

Consider an array of 2D vertices:

`data = numpy.zeros(4, dtype = [("position", np.float32, 2)
    ("colour", np.float32, 4)]`

To tell the vertex shader that it needs to handle vertices where postion is a tuple of 2 floating-point integers we need to use attributes.

`attribute vec2 position;
 attribute vec4 colour;
 void main()
 {
     gl_Position = vec4(position, 0.0, 1.0);
 }`

The uniform, a constant value, can also be passed to the vertex shader. So, if we wanted to scale all the vertices by a constant value:

`uniform float scale;
 attribute vec2 position;
 attribute vec4 colour;
 void main()
 {
     gl_Position = vec4(position*scale, 0.0, 1.0);
 }`

Let's say we would like to pass the vertex colour to the fragment shader:

`uniform float scale;
 attribute vec2 position;
 attribute vec4 colour;
 varying vec4 v_colour
 void main()
 {
     gl_Position = vec4(position*scale, 0.0, 1.0);
     v_colour = colour
 }`

and so the fragment shdaer would be given by the following:

`varying vec4 v_colour
 void main()
 {
     gl_FragColor = v_colour
 }`

Any varying value is interpolated between the vertices of the item in question.

## State of the Union

We need to access the OpenGL library from within Python itself. It is easiest to use a set of bindings, such as those founf in PyOpenGL. The glumpy library is useful for scientific visualisation and uses the power from GPUs through the OpenGL library to display large datasets.

## Preliminaries

### Normalise Device Coordinates

Coordinates are normalised to -1<x<1, -1<y<1 and -1<z<1. These are know as normalised device coordinates (NDCs). x coordinates increase from left to right, and y coordinates from bottom to top.

### Triangulation

Finding a set of triangles which cover a given surface. Quality of this process is measured by its closeness to the actual shape, the homogeneity of the triangles and the number of triangles used. There exist many algorithms and tools that do this automatically for you. For a simple square it makes sense to used two triangles: one has vertices at (-1,1),(1,1) and (-1,-1) and the other has vertices at (1,1), (-1,-1) and (1,-1). These two triangles have the (-1,-1) and (1,1) vertices in common, so the whole square can be described using the following:

V<sub>0</sub> = (-1,1);
V<sub>1</sub> = (1,1);
V<sub>2</sub> = (-1,-1);
V<sub>3</sub> = (1,-1)

So our square can be described using the triangles (V<sub>0</sub>,V<sub>1</sub>,V<sub>2</sub>) and (V<sub>1</sub>,V<sub>2</sub>,V<sub>3</sub>).

### GL Primitives

Lines can be drawn using GL_LINES, broken lines using GL_LINE_STRIP, and closed broken lines using GL_LINE_LOOP. Triangles can be drawn either using GL_TRIANGLES or GL_TRIANGLE_STRIP (if using an implicit structure i.e. given a set of vertices V<sub>i</sub> GL_TRIANGLES_STRIP produces a triangle of vertices V<sub>i</sub>,V<sub>i+1</sub> and V<sub>i+2</sub>.

### Interpolation

The triangle is chosen to be the only surface primative. This choice is not arbitrary: the triangle is chosen because it allows intuitive interpolation for any point inside the triangle.

Rasterisation requires OpenGL to generate fragments inside of the triangle as well as interpolate values (such as colour). See **barycentric interpolation**.

### The Hard Way

Left for now.

### The Easy Way

Glumpy uses three main modules:
* **app**: Application Layer. This is responsible for opening a window and handling user events like the mouse and keyboard interactions.
* **gloo**: Object Oriented Layer. This is responsible for handling shader programs and syncing CPU/GPU data through the NumPy interface.
* **graphics**: Graphic Layer. This provides higher-level common objects such as text, collections and widgets.